In [1]:
!pip install tinydb 
!pip install pandas_ta
!pip install python-binance
# !pip install yfinance
!pip install mplfinance

  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218925 sha256=6b3766e4da897c0c35b939f0bf8114d4abba2a738d864e9159d8e360cb29d21a
  Stored in directory: c:\users\stb\appdata\local\pip\cache\wheels\54\4a\75\06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas-ta


In [29]:
import os
import sys
import json
# from dotenv import load_dotenv
import asyncio
import threading
import datetime
from multiprocessing import Pool, cpu_count
import matplotlib.dates as mpl_dates
# Other modules
import pandas as pd
import pandas_ta as ta
from pandas.io import sql
import numpy as np
import matplotlib.pyplot as plt

# python-binance module
from binance.client import Client
from binance import BinanceSocketManager
from binance.enums import *

# BINANCE_KEY = os.getenv('BINANCE_KEY')
# BINANCE_SECRET = os.getenv('BINANCE_SECRET')
BINANCE_KEY = ''
BINANCE_SECRET = ''

# connect binance
client = Client(BINANCE_KEY, BINANCE_SECRET)


In [30]:
def get_candlestick_one_token(symbol):
    df1 = pd.DataFrame(client.get_historical_klines(symbol,
                                                       client.KLINE_INTERVAL_5MINUTE,
                                                       "1 day ago UTC"
                                                       ))
    df1.columns = ["Date", "Open", "High", "Low", "Close", "Volume",
                  "Close_time", "Qav", "Num_trades", "Taker_base_vol",
                  "Taker_quote_vol", "Is_best_match"]
    df_simple = df1[["Date", "Open", "High", "Low", "Close", "Volume"]]
    df_simple[["Open", "High", "Low", "Close", "Volume"]] = df_simple[["Open", "High", "Low", "Close", "Volume"]].apply(pd.to_numeric)
    df_simple['Date_readonly'] = df_simple['Date'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
    df_simple['Date'] = pd.to_datetime(df_simple['Date'])
    df_simple['Date'] = df_simple['Date'].apply(mpl_dates.date2num)
    df_simple['Date'] = df_simple['Date']*1000000
    
    return df_simple
    

In [31]:
def analyse_rsi(df):
    rsi = ta.rsi(df['Close'])
    last_rsi =  rsi.iloc[-1]
    return last_rsi

In [32]:
def get_rsi(symbol):
    df = get_candlestick_one_token(symbol)
    return analyse_rsi(df)
    

In [33]:
get_rsi("NEARUSDT")

/home/lenovo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/lenovo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/lenovo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

53.76489614696657